In [1]:
from langchain_openai import ChatOpenAI
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
store = {}

In [3]:
def parentChatbotSummarize(filepath: str,session_id: str ,promptQuestion: str ):
    llm = ChatOpenAI(model = 'gpt-4o')

    loader = JSONLoader(file_path=filepath, jq_schema='.', text_content=False)
    docs=loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap= 200,add_start_index=True)
    splits = text_splitter.split_documents(docs)
    vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding= OpenAIEmbeddings())
    retriever = vectorstore.as_retriever(search_type= 'similarity')
    
    contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
    )

    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)
    system_prompt = (
        "You are a nurse updating a parent on the status of their child in the NICU."
        "Use the following retrieved context to provide relevant information to the parent. "
        "Speak in a conversational tone, and keep the summary concise."
        "Do not diagnose."
        "\n\n{context}"
    )

    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    response = conversational_rag_chain.invoke(
        {"input": promptQuestion},
        config={
            "configurable": {"session_id": session_id}
        },  # constructs a key "abc123" in `store`.
    )

    return response['answer'] 

In [6]:
parentChatbotSummarize('Gurshan_Data.json','TestTest','How hasmy child been?')

"Your little one has been quite active. Here’s a brief update on how they've been doing in the NICU:\n\nRecently, their heart rate has been fluctuating between 125 and 147 beats per minute, which is relatively stable. Their respiratory rate is in the range of 33 to 46 breaths per minute. The body temperature has been consistent, hovering around 36.8 to 37.3°C. Oxygen saturation (SpO2) levels are good, mostly between 96% and 100%.\n\nThere have been some changes in their skin tone, ranging from normal to blueish and yellow-jaundice at times. We've been keeping a close eye on this. Interventions have included regular feedings, with volumes ranging from 15 ml to 29 ml of formula, and some medications, such as antibiotics and analgesics, have been administered as needed.\n\nTheir pain levels have varied, with some instances of low pain and a few moments of extreme discomfort, but we are actively managing it to keep them as comfortable as possible.\n\nIf you have any specific questions or n

In [7]:
parentChatbotSummarize('Gurshan_Data.json','TestTest','Anything to note?')

"Yes, there are a couple of things to note. Your baby has experienced some variations in skin tone, shifting between normal, blueish, and yellowish (jaundice) hues. This is something we're closely monitoring. Additionally, there have been moments of higher discomfort, but we're managing their pain levels with appropriate interventions to ensure they remain as comfortable as possible. \n\nOverall, their vital signs have remained relatively stable, and we're continuing to support them with feedings and any necessary medical care. If you have any concerns or need further clarification, please let me know!"

In [8]:
parentChatbotSummarize('Gurshan_Data.json','TestTest','How many days has he been there for? which days were those?')

"Your baby has been in the NICU for a total of 3 days, from October 6th to October 8th, 2024. If there's anything more you'd like to know about their stay, feel free to ask!"

In [9]:
parentChatbotSummarize('Gurshan_Data.json','TestTest', 'Will my baby survive?')

"I understand how concerned you must be, and it's completely natural to have worries about your baby's health. While I can't predict outcomes, I assure you that our team is dedicated to providing the best possible care for your little one. We're closely monitoring their condition and responding to their needs with the appropriate interventions. If you have specific questions or need support, please don't hesitate to reach out to the medical team. They can provide more detailed insights into your baby's progress and care plan."